In [1]:
import glob
import numpy
import pandas
import seaborn
import matplotlib.pyplot as plt
from tqdm import tqdm
import multiprocessing as mp
import os
from build import build_model

In [2]:
from IPython.display import display, HTML, Math, Markdown
display(HTML("<style>.container { width:95% !important; }</style>"))

%load_ext autoreload
%autoreload 2

In [3]:
!bash report.sh
organisms = set(pandas.read_csv("biomass_constrained.txt",index_col=0,header=None).index.to_list())
done = {i.split(".csv")[0] for i in os.listdir("./cases/fluxes/low_iron/")}

In [4]:
run_for = ["Bartonella_quintana_Toulouse","Ochrobactrum_anthropi_ATCC_49188"]
len(run_for)

2

In [5]:
from diets import load_model,constrain_exchanges,get_bounds
from overflow import get_overflow
import coralme

conditions = ["low"]
exchanges = ["EX_fe2(e)","EX_fe3(e)","TS_2fe2s_c","TS_4fe4s_c","TS_fe3_c"]
metabolite = "iron"

for c in conditions:
    name = "{}_{}".format(c,metabolite)
    fluxdir = "./cases/fluxes/{}".format(name)
    overflowdir = "./cases/overflow/{}".format(name)
    if not os.path.isdir(fluxdir):
        os.makedirs(fluxdir)
        os.makedirs(overflowdir)

def run(org):
    model = coralme.io.pickle.load_pickle_me_model("./me-models/{}/MEModel-BIO-{}-ME-TS.pkl".format(org,org))
    get_overflow(model).to_csv("./cases/overflow/base/{}.csv".format(org))
    sol0 = model.solution
    df0 = sol0.to_frame()
    # Exchanges
    rlist = [r for r in exchanges if r in model.reactions]
    # Run for all conditions
    for condition in conditions:
        name = "{}_{}".format(c,metabolite)
        dct = get_bounds(sol0,rlist,multiplier=0.01)
        constrain_exchanges(model,dct)
        model.optimize(max_mu = model.solution.objective_value * 2)
        model.solution.to_frame().to_csv("./cases/fluxes/{}/{}.csv".format(name,org))
        get_overflow(model).to_csv("./cases/overflow/{}/{}.csv".format(name,org))
        

In [6]:
NP = min([10,len(run_for)])
pool = mp.Pool(NP,maxtasksperchild=1)
pbar = tqdm(total=len(run_for),position=0,leave=True)
pbar.set_description('Building ({} threads)'.format(NP))
def collect_result(result):
    pbar.update(1)
    
for org in run_for:
    args = ([org])
    pool.apply_async(run,args, callback=collect_result)
pool.close()
pool.join()

Building (2 threads):   0%|                               | 0/2 [00:00<?, ?it/s]

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16
Read LP format model from file /tmp/tmpk9tucq13.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Read LP format model from file /tmp/tmpyswjazwj.lp
Reading time = 0.00 seconds
: 832 rows, 1788 columns, 7876 nonzeros
Read LP format model from file /tmp/tmpcu5gxy0o.lp
Reading time = 0.00 seconds
: 854 rows, 1786 columns, 7706 nonzeros
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16
Read LP format model from file /tmp/tmphzf4han6.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Iteration	 Solution to check	Solver Status
---------	------------------	-------------
Read LP format model from file /tmp/tmpd_q3yjob.lp
Reading time = 0.00 seconds
: 1301 rows, 3132 columns, 13778 nonzeros
Read LP format model from file /tmp/tmp_6p0hnvj.lp
Reading time = 0.00 seconds
: 1316 rows, 3130 columns, 13608 nonzeros
        1	0.0044593176008627	N